In [1]:
# New packages introduced in this tutorial (run this if you need to install these)
using JuMP, HiGHS
using Plots; plotly();
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
using FileIO
using PlotlyBase
using PlotlyKaleido
ENV["COLUMNS"]=120;

In [2]:
##########Single-time period, simple generator constraints################

datadir = joinpath("ed_data") 
# Note: joinpath is a good way to create path reference that is agnostic
# to what file system you are using (e.g. whether directories are denoted 
# with a forward or backwards slash).
gen_info = CSV.read(joinpath(datadir,"Generators_data.csv"), DataFrame);
fuels = CSV.read(joinpath(datadir,"Fuels_data.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Demand.csv"), DataFrame);
gen_variable = CSV.read(joinpath(datadir,"Generators_variability.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gen_info, fuels, loads, gen_variable]
    rename!(f,lowercase.(names(f)))
end
     

In [3]:
# Keep only the columns relevant to our ED model 
# (We'll come back to other columns in our unit commitment notebooks)
select!(gen_info, 1:26, :stor) 
gen_df = outerjoin(gen_info,  fuels, on = :fuel) # load in fuel costs and add to data frame
rename!(gen_df, :cost_per_mmbtu => :fuel_cost)   # rename column for fuel cost
gen_df[ismissing.(gen_df[:,:fuel_cost]), :fuel_cost] .= 0

# create "is_variable" column to indicate if this is a variable generation source (e.g. wind, solar)
# Note: Julia's strict vectorization syntax requires us to create a new variable using the '!' indexing operator
# and the '.=' broadcasting assignment. See: https://julia.guide/broadcasting
gen_df[!, :is_variable] .= false
gen_df[in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic"]).(gen_df.resource),
    :is_variable] .= true;

# create full name of generator (including geographic location and cluster number)
#  for use with variable generation dataframe
gen_df.gen_full = lowercase.(gen_df.region .* "_" .* gen_df.resource .* "_" .* string.(gen_df.cluster) .* ".0");

# remove generators with no capacity (e.g. new build options that we'd use if this was capacity expansion problem)
gen_df = gen_df[gen_df.existing_cap_mw .> 0,:];


In [4]:
df = gen_df[1:5,1:5]
     

df = copy(gen_df)
select!(df, 1:5)
df = df[1:5,:]
     

stack(df, 
    [:existing_cap_mw, :num_units], 
    variable_name=:var,
    value_name=:val)

Row,r_id,resource,region,var,val
,Int64?,String?,String15?,String,Float64?
1,1,biomass,WEC_SDGE,existing_cap_mw,21.4
2,2,hydroelectric_pumped_storage,WEC_SDGE,existing_cap_mw,42.0
3,3,natural_gas_fired_combined_cycle,WEC_SDGE,existing_cap_mw,607.1
4,4,natural_gas_fired_combined_cycle,WEC_SDGE,existing_cap_mw,30.0
5,5,natural_gas_fired_combined_cycle,WEC_SDGE,existing_cap_mw,49.4
6,1,biomass,WEC_SDGE,num_units,20.0
7,2,hydroelectric_pumped_storage,WEC_SDGE,num_units,2.0
8,3,natural_gas_fired_combined_cycle,WEC_SDGE,num_units,1.0
9,4,natural_gas_fired_combined_cycle,WEC_SDGE,num_units,1.0


In [5]:
gen_variable.hour = mod.(gen_variable.hour .- 9, 8760) .+ 1 
sort!(gen_variable, :hour)
loads.hour = mod.(loads.hour .- 9, 8760) .+ 1
sort!(loads, :hour);

In [6]:
describe(gen_variable)
     

gen_variable_long = stack(gen_variable, 
                        Not(:hour), 
                        variable_name=:gen_full,
                        value_name=:cf);
# Now we have a "long" dataframe; 
# let's look at the first 6 entries of a wind resource for example
first(gen_variable_long[gen_variable_long.gen_full.=="wec_sdge_onshore_wind_turbine_1.0",:],6)

Row,hour,gen_full,cf
,Int64,String,Float64
1,1,wec_sdge_onshore_wind_turbine_1.0,0.1694
2,2,wec_sdge_onshore_wind_turbine_1.0,0.1658
3,3,wec_sdge_onshore_wind_turbine_1.0,0.1502
4,4,wec_sdge_onshore_wind_turbine_1.0,0.1455
5,5,wec_sdge_onshore_wind_turbine_1.0,0.0932
6,6,wec_sdge_onshore_wind_turbine_1.0,0.0717


In [7]:
hr = 2416  # pick 4pm on a spring day
loads_single = loads[loads[:,:hour] .== hr, Not(:hour)];
var_cf_single = gen_variable_long[
    gen_variable_long.hour .== hr, 
    Not(:hour)]

Row,gen_full,cf
,String,Float64
1,wec_sdge_biomass_1.0,1.0
2,wec_sdge_hydroelectric_pumped_storage_1.0,1.0
3,wec_sdge_natural_gas_fired_combined_cycle_1.0,1.0
4,wec_sdge_natural_gas_fired_combined_cycle_2.0,1.0
5,wec_sdge_natural_gas_fired_combined_cycle_3.0,1.0
6,wec_sdge_natural_gas_fired_combined_cycle_4.0,1.0
7,wec_sdge_natural_gas_fired_combustion_turbine_1.0,1.0
8,wec_sdge_natural_gas_fired_combustion_turbine_2.0,1.0
9,wec_sdge_natural_gas_fired_combustion_turbine_3.0,1.0


In [8]:
#=
Function to solve economic dispatch problem (single-time period, single-zone)
Inputs:
    gen_df -- dataframe with generator info
    loads  -- dataframe with load info
    gen_variable -- capacity factors of variable generators (in "long" format)
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function economic_dispatch_single(gen_df, loads, gen_variable)
    ED = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # A set of all variable generators
    G_var = gen_df[gen_df[!,:is_variable] .== 1,:r_id] 
      # A set of all non-variable generators
    G_nonvar = gen_df[gen_df[!,:is_variable] .== 0,:r_id]
      # Set of all generators
    G = gen_df.r_id
    # Extract some parameters given the input data
      # Generator capacity factor time series for variable generators
    gen_var_cf = innerjoin(gen_variable, 
                    gen_df[gen_df.is_variable .== 1 , 
                        [:r_id, :gen_full, :existing_cap_mw]], 
                    on = :gen_full)
        
    # Decision variables   
    @variables(ED, begin
        GEN[G]  >= 0     # generation
        # Note: we assume Pmin = 0 for all resources for simplicty here
    end)
                
    # Objective function
    @objective(ED, Min, 
        sum( (gen_df[i,:heat_rate_mmbtu_per_mwh] * gen_df[i,:fuel_cost] +
            gen_df[i,:var_om_cost_per_mwh]) * GEN[i] 
                        for i in G_nonvar) + 
        sum(gen_df[i,:var_om_cost_per_mwh] * GEN[i] 
                        for i in G_var)
    )

    # Demand constraint
    @constraint(ED, cDemand, 
        sum(GEN[i] for i in G) == loads[1,:demand])

    # Capacity constraint (non-variable generation)
    for i in G_nonvar
        @constraint(ED, GEN[i] <= gen_df[i,:existing_cap_mw])
    end

    # Variable generation capacity constraint
    for i in 1:nrow(gen_var_cf)
        @constraint(ED, GEN[gen_var_cf[i,:r_id] ] <= 
                        gen_var_cf[i,:cf] *
                        gen_var_cf[i,:existing_cap_mw])
    end

    # Solve statement (! indicates runs in place)
    optimize!(ED)

    # Dataframe of optimal decision variables
    solution = DataFrame(
        r_id = gen_df.r_id,
        resource = gen_df.resource,
        gen = value.(GEN).data
        )

    # Return the solution and objective as named tuple
    return (
        solution = solution, 
        cost = objective_value(ED),
    )
end

economic_dispatch_single (generic function with 1 method)

In [9]:
solution = economic_dispatch_single(gen_df, loads_single, var_cf_single);
solution.solution

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
1 rows, 25 cols, 25 nonzeros
1 rows, 22 cols, 22 nonzeros
Presolve : Reductions: rows 1(-25); columns 22(-3); elements 22(-28)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1(2482) 0s
          1     5.8477778205e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1
Objective value     :  5.8477778205e+04
HiGHS run time      :          0.01


Row,r_id,resource,gen
,Int64?,String?,Float64
1,1,biomass,21.4
2,2,hydroelectric_pumped_storage,42.0
3,3,natural_gas_fired_combined_cycle,607.1
4,4,natural_gas_fired_combined_cycle,30.0
5,5,natural_gas_fired_combined_cycle,49.4
6,6,natural_gas_fired_combined_cycle,570.0
7,7,natural_gas_fired_combined_cycle,336.0
8,8,natural_gas_fired_combined_cycle,336.0
9,9,natural_gas_fired_combustion_turbine,0.0


In [10]:
supply_curve = leftjoin(gen_df,
                var_cf_single, 
                on = :gen_full)
supply_curve[!, :varcost] .= 0.0
supply_curve[!, :cap] .= 0.0

# Store varcost of non-variable generators:
I = supply_curve[!,:is_variable] .== 0  # `I` contains indexes to the non-variable generators
supply_curve[I,:varcost] .= 
    supply_curve[I,:heat_rate_mmbtu_per_mwh] .* 
        supply_curve[I,:fuel_cost] .+
    supply_curve[I,:var_om_cost_per_mwh]

# Calculate available capacity for each generator
supply_curve[I,:cap] = supply_curve[I,:existing_cap_mw]

# Store varcost of variable generators (in this case, 0)
I = (supply_curve[:,:is_variable] .== 1) # `I` contains indexes to the variable generators
supply_curve[I,:varcost] = supply_curve[I,:var_om_cost_per_mwh]

# Calculate available capacity for each generator
# (adjusted for variable generation)
supply_curve[I,:cap] = supply_curve[I,:existing_cap_mw] .* supply_curve[I,:cf]

sort!(supply_curve, :varcost);

In [11]:
rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])

p = plot(nrow(supply_curve))

marg_gen = 0
marg_price = 0
x = 0
for i in 1:nrow(supply_curve)
    plot!(p, rectangle(supply_curve[i,:cap],
            supply_curve[i,:varcost],x,0), opacity=.5, 
            label = "")
    if (x < loads_single[1,:demand]) && (x + supply_curve[i,:cap] > loads_single[1,:demand])
        marg_gen = i
        marg_price = supply_curve[i,:varcost]
    end
    x = x + supply_curve[i,:cap]
end 
vline!([loads_single[1,:demand]],linecolor="black",linewidth=5,
label = "demand")
hline!([marg_price],linecolor="blue",linewidth=5,
label = "SRMC")
title!("Supply and demand curves")
xlabel!("Capacity")
ylabel!("Marginal cost")
p

LoadError: UndefVarError: Shape not defined

In [12]:
supply_curve[marg_gen,:]

LoadError: BoundsError: attempt to access 25×34 DataFrame at index [0, :]

In [13]:
sol_gen = combine(groupby(solution.solution, :resource), 
            :gen => sum)
# We use the PrettyTable package here to round numbers when displaying
pretty_table(sol_gen, formatters = ft_printf("%5.3f")) 

┌──────────────────────────────────────┬──────────┐
│                             resource │  gen_sum │
│                              String? │  Float64 │
├──────────────────────────────────────┼──────────┤
│                              biomass │   21.400 │
│         hydroelectric_pumped_storage │   42.000 │
│     natural_gas_fired_combined_cycle │ 1928.500 │
│ natural_gas_fired_combustion_turbine │  136.755 │
│                 onshore_wind_turbine │   65.411 │
│                  small_hydroelectric │    3.185 │
│                   solar_photovoltaic │  284.750 │
└──────────────────────────────────────┴──────────┘


In [14]:
# we use the copy function to create a copy of the dataframe (as opposed to working with the original)
sol_gen_btm = copy(sol_gen)

# We'll use the same variability profile for BTM solar as the utility-scale solar here (in reality, they would probably differ)
btm = DataFrame(resource = ["solar_photovoltaic_btm"], gen_sum = var_cf_single[var_cf_single.gen_full .== "wec_sdge_solar_photovoltaic_1.0",:cf] * 600)
append!(sol_gen_btm, btm)
pretty_table(sol_gen_btm, formatters = ft_printf("%5.3f")) # We use the PrettyTable package to round numbers when displaying
     

┌──────────────────────────────────────┬──────────┐
│                             resource │  gen_sum │
│                              String? │  Float64 │
├──────────────────────────────────────┼──────────┤
│                              biomass │   21.400 │
│         hydroelectric_pumped_storage │   42.000 │
│     natural_gas_fired_combined_cycle │ 1928.500 │
│ natural_gas_fired_combustion_turbine │  136.755 │
│                 onshore_wind_turbine │   65.411 │
│                  small_hydroelectric │    3.185 │
│                   solar_photovoltaic │  284.750 │
│               solar_photovoltaic_btm │  341.700 │
└──────────────────────────────────────┴──────────┘
